In [24]:
%load_ext memory_profiler
%memit
import pandas as pd
import numpy as np


data = np.zeros((100000, 5))
data = pd.DataFrame(data, columns=['col%d'%i for i in range(data.shape[1])])
data.to_parquet('tmp.parquet')

data = np.zeros((100000, 5))
data = pd.DataFrame(data, columns=['col%d'%i for i in range(data.shape[1])])
data.to_csv('tmp.csv', index=False)

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
peak memory: 301.79 MiB, increment: 0.00 MiB


In [59]:
%memit
from dask import dataframe as DF

df = DF.read_parquet('tmp.parquet')
df = df.repartition(partition_size="100KB")
# part_path = ['tmpcsv/part.{}'.format(i) for i in range(df.npartitions)]

def name_fn(i):
    return 'mypart.{}'.format(i)

df.to_parquet(path='tmpparquet', write_metadata_file=False, name_function=name_fn)

peak memory: 331.49 MiB, increment: 0.00 MiB


(None,)

In [26]:
data = DF.read_parquet('./tmpp/part.0.parquet')
data = data.compute()
import sys
sys.getsizeof(data)
print(data.shape)
data

(2439, 5)


,col0,col1,col2,col3,col4
0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
2434,0.0,0.0,0.0,0.0,0.0
2435,0.0,0.0,0.0,0.0,0.0
2436,0.0,0.0,0.0,0.0,0.0
2437,0.0,0.0,0.0,0.0,0.0


In [51]:
%memit
from dask import dataframe as DF

df = DF.read_csv('tmp.csv')
df = df.repartition(partition_size="100KB")
part_path = ['tmpcsv/part.{}'.format(i) for i in range(df.npartitions)]
df.to_csv(part_path, index=False)

peak memory: 450.36 MiB, increment: 0.00 MiB


['/home/kangz1/Documents/DLCache/fs-kv-store/tmpcsv/part.0',
 '/home/kangz1/Documents/DLCache/fs-kv-store/tmpcsv/part.1',
 '/home/kangz1/Documents/DLCache/fs-kv-store/tmpcsv/part.2',
 '/home/kangz1/Documents/DLCache/fs-kv-store/tmpcsv/part.3',
 '/home/kangz1/Documents/DLCache/fs-kv-store/tmpcsv/part.4',
 '/home/kangz1/Documents/DLCache/fs-kv-store/tmpcsv/part.5',
 '/home/kangz1/Documents/DLCache/fs-kv-store/tmpcsv/part.6',
 '/home/kangz1/Documents/DLCache/fs-kv-store/tmpcsv/part.7',
 '/home/kangz1/Documents/DLCache/fs-kv-store/tmpcsv/part.8',
 '/home/kangz1/Documents/DLCache/fs-kv-store/tmpcsv/part.9',
 '/home/kangz1/Documents/DLCache/fs-kv-store/tmpcsv/part.10',
 '/home/kangz1/Documents/DLCache/fs-kv-store/tmpcsv/part.11',
 '/home/kangz1/Documents/DLCache/fs-kv-store/tmpcsv/part.12',
 '/home/kangz1/Documents/DLCache/fs-kv-store/tmpcsv/part.13',
 '/home/kangz1/Documents/DLCache/fs-kv-store/tmpcsv/part.14',
 '/home/kangz1/Documents/DLCache/fs-kv-store/tmpcsv/part.15',
 '/home/kangz1/Doc

In [70]:
%memit
import pickle

for part in df.partitions:
    tmp = pickle.dumps(part.compute())
    print(tmp)
    break

peak memory: 421.62 MiB, increment: 0.00 MiB
b'\x80\x04\x95B\x01\x00\x00\x00\x00\x00\x00\x8c\x11pandas.core.frame\x94\x8c\tDataFrame\x94\x93\x94)\x81\x94}\x94(\x8c\x04_mgr\x94\x8c\x1epandas.core.internals.managers\x94\x8c\x0cBlockManager\x94\x93\x94\x8c\tfunctools\x94\x8c\x07partial\x94\x93\x94\x8c\x1cpandas.core.internals.blocks\x94\x8c\tnew_block\x94\x93\x94\x85\x94R\x94(h\x0e)}\x94\x8c\x04ndim\x94K\x02sNt\x94b\x8c\x15numpy.core.multiarray\x94\x8c\x0c_reconstruct\x94\x93\x94\x8c\x05numpy\x94\x8c\x07ndarray\x94\x93\x94K\x00\x85\x94C\x01b\x94\x87\x94R\x94(K\x01K\x05M\x87\t\x86\x94h\x17\x8c\x05dtype\x94\x93\x94\x8c\x02f8\x94\x89\x88\x87\x94R\x94(K\x03\x8c\x01<\x94NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00t\x94b\x89B\x18}\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0